In [46]:
import numpy as np
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split

In [47]:
def parse_data(fpath):
    data = np.genfromtxt(fpath, skip_header=7)
    X, y = data[:, :-1], data[:, -1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    return X_train, X_test, y_train, y_test

In [48]:
X_train, X_test, y_train, y_test = parse_data("/Users/ChesterHuynh/classes/dldo467/dldo-hw/hw2/stable5.txt")

In [49]:
class My_Net(nn.Module): 
    def __init__(self, input_size, num_neurons):
        super(My_Net, self).__init__()
        #feed forward layers
        self.layer_1 = nn.Linear(input_size, num_neurons)
        self.layer_2 = nn.Linear(num_neurons, num_neurons)
        self.layer_3 = nn.Linear(num_neurons, 1)

        #activations
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid() #Use sigmoid to convert the output into range (0,1)
        
    def forward(self, input_data):
        out = self.layer_1(input_data)
        out = self.relu(out)
        out = self.layer_2(out)
        out = self.relu(out)
        out = self.layer_3(out)
        out = self.sigmoid(out).view(-1)
        return out

In [80]:
input_size = X_train.shape[1]

net=My_Net(input_size=input_size, num_neurons=50)

#Standard cross entropy loss for classification tasks
loss=nn.BCELoss()

#Define the optimizer. Here we use Adam optimizer.
opt=torch.optim.Adam(net.parameters(),lr=1)

In [81]:
Xtrain = torch.Tensor(X_train)
Xtest = torch.Tensor(X_test)
ytrain = torch.Tensor(y_train)
ytest = torch.Tensor(y_test)

In [82]:
train = torch.utils.data.TensorDataset(Xtrain, ytrain)
test = torch.utils.data.TensorDataset(Xtest, ytest)

train_loader = torch.utils.data.DataLoader(train, batch_size=128, num_workers=2)
test_loader = torch.utils.data.DataLoader(test, batch_size=128, shuffle=False,num_workers=2)

In [83]:
def train_eval(verbose=1):
    correct = 0
    total = 0
    loss_sum = 0
    num_batches = 0
    for inputs, labels in train_loader:
        outputs = net(inputs)
        predicted = outputs.data>0.5
        total += labels.size(0)
        correct += (predicted.int() == labels.int()).sum()
        loss_sum  += loss(outputs,labels).item()
        num_batches += 1

    if verbose:
        print('Train accuracy: %f %%' % (100 * correct.item() / total))
    return loss_sum/num_batches, correct.item() / total

def test_eval(verbose=1):
    correct = 0
    total = 0
    loss_sum = 0
    num_batches = 0
    for inputs, labels in test_loader:
        outputs = net(inputs)
        predicted = outputs.data>0.5
        total += labels.size(0)
        correct += (predicted.int() == labels.int()).sum()
        loss_sum  += loss(outputs,labels).item()
        num_batches += 1

    if verbose:
        print('Test accuracy: %f %%' % (100 * correct.item() / total))
    return loss_sum/num_batches, correct.item() / total

In [84]:
#initialize the network using Xavier initialization.
def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight.data)

net.apply(weights_init)

My_Net(
  (layer_1): Linear(in_features=10, out_features=50, bias=True)
  (layer_2): Linear(in_features=50, out_features=50, bias=True)
  (layer_3): Linear(in_features=50, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

In [85]:
import time

epochs=25;
train_loss_store = []
train_acc_store = []
test_loss_store = []
test_acc_store = []


for epoch in range(epochs):
    time1=time.time()
    print('In epoch %i : '%(epoch+1))
    for i, (x,y) in enumerate(train_loader,0):
        #set the gradients to zero initially for each batch
        opt.zero_grad()
        outputs = net(x)
        l=loss(outputs, y)
        print(l)
        l.backward()
        opt.step()
    
    l_temp, acc_temp = train_eval()
    train_loss_store.append(l_temp)
    train_acc_store.append(acc_temp)

    l_temp, acc_temp = test_eval()
    test_loss_store.append(l_temp)
    test_acc_store.append(acc_temp)

    time2=time.time()
    print("Time lapse: %f secs" %round((time2-time1),2))

In epoch 1 : 
tensor(1.4345, grad_fn=<BinaryCrossEntropyBackward>)
tensor(-47.2750, grad_fn=<BinaryCrossEntropyBackward>)
tensor(-49.2178, grad_fn=<BinaryCrossEntropyBackward>)
tensor(-45.3321, grad_fn=<BinaryCrossEntropyBackward>)
tensor(-48.5702, grad_fn=<BinaryCrossEntropyBackward>)
tensor(-46.5365, grad_fn=<BinaryCrossEntropyBackward>)
